# 다형성과 유형클래스

## 매개변수 다형성

다형유형의 정의를 **특정 형식의 유형을 임의의 유형을 이용해서 생성할 수 있도록 하는 기법**으로 이해할 수 있다. 
이전에 살펴보았던 리스트, 튜플, 함수 유형 모두 임의의 유형을 사용하여 동일한 형식의 새로운 유형을 생성한다.

* 리스트 유형: `[Int]`, `[Float]`, `[String]`, `[Bool]`, 등등.

* 튜플 유형: `(Int, String)`, `(Float, Bool)`, `(String, Char)`, 등등.

* 함수 유형: `Int -> Float`, `(Int, Int) -> Int`, `[Bool] -> Int`, 등등.

다형유형은 하스켈이 지원하는 다형성polymorphism의 핵심이며, 
앞서 설명한 이유로 해서 **매개변수 다형성**parametric polymorphism이라고도 불린다. 

`[a]`, `(a, b)`, `a -> b` 등의 경우처럼 유형 변수type variable를 
임의의 유형을 인자로 받을 수 있는 일종의 매개변수로 간주한다는 의미에서 매개변수 다형성이라 부른다고 이해할 수 있다. 

### 다형 유형의 엄밀한 적용법

`t :: [a]`, `t :: (a, b)`, `t :: a -> b`의 엄밀한 의미는 다음과 같다.

* `t :: forall a. [a]`
    * 임의의 유형 `A`에 대해 `t A :: [A]`가 성립한다. 
        즉, `t A`는 `A` 유형의 값들로 이루어진 리스트이다. 
    * 예를 들어, `t Int`는 정수들의 리스트이어야 한다. 
* `t :: forall a b. (a, b)`
    * 임의의 유형 `A, B`에 대해 `t A B :: (A, B)`가 성립한다. 
        즉, `t A B`는 `A` 유형의 값과 `B` 유형의 값으로 이루어진 길이가 2인 튜플이다. 
    * 예를 들어, `t Int Int`는 두 개의 정수로 이루어진 순선쌍이어야 한다. 
* `t :: forall a b. a -> b`
    * 임의의 유형 `A, B`에 대해 `t A B :: A -> B`가 성립한다. 
        즉, `t A B`는 `A` 유형에서 `B` 유형으로 가는 함수이다. 
    * 예를 들어, `t (Int,Int) Int`는 길이가 2인 정수들의 순선쌍에 적용하면 새로운 정수를 계산해야 한다. 

### 유형 추론

다형유형의 함수를 사용할 때 앞서 설명한 대로 사용되는 특정 유형을 지정하는 일은 매우 귀찮은 일이다.
하지만 일반적으로 그럴 필요가 없다. 
하스켈이 사용되는 값의 유형을 기본적으로 스스로 유추할 수 있기 때문이며, 
이를 **유형 추론**type inference이라 부른다. 

**주의:** 유형 추론이 경우에 따라 불가능 할 수 있다. 
예를 들어, 공 리스트 `[]`와 정수 `1, 2, 3` 등의 유형이 명확하지 않을 수 있다. 
공 리스트는 임의의 유형의 리스트일 수 있으며, `1, 2, 3` 등의 숫자는 `Int`, `Integer` 등이 될 수 있기 때문이다.
하스켈은 유형에 매우 엄격하기 때문에 `Int`를 기대하는 함수에 `Integer`를 적용하면 오류가 발생한다. 

#### 예제

리스트에 포함된 항목들의 순서를 바꾸는 `reverse` 함수를 살펴보자. 
`reverse`는 임의의 유형의 리스트에 대해 작동하며, 유형은 아래와 같다. 

In [1]:
:t reverse

reverse :: forall a. [a] -> [a]

`reverse` 함수를 리스트 `[True, True, False, False]`와 함께 적용하면 아래와 같이 작동한다. 

In [2]:
reverse [True, False, False]

[False,False,True]

엄밀히 따지면 `[True, False, False]`의 유형 `[Bool]`에 사용된 `a`를 `reverse`가 알아야 한다.
하지만 `[True, False, False]`를 보고서 `[a]`에 사용된 `a`가 `Bool`임을 추론할 수 있기에 
굳이 `a`에 대한 정보를 입력하지 않아도 되는 것이다. 

**주의:** 경우에 따라 유형을 명시해야 하는 경우가 발생한다. 그럴 경우 아래와 같이 하면 된다.

In [3]:
reverse [True, False, False] :: [Bool]

[False,False,True]

## 도구 다형성

다형성과 관련해서 하스켈이 제공하는 또다른 기능이 하나의 함수를 여러 유형에 동일한 이름으로 
작동하도록 해주는 것이다.

#### 예제: 덧셈

예를 들어, 덧셈 연산자 `+`는 정수들의 덧셈뿐만 아니라 실수들의 덧셈도 계산한다. 

In [4]:
2 + 3

5

In [5]:
2.0 + 3.0

5.0

#### 예제: 동치비교

이 기능이 가장 중요한 연산자 중에 하나가 동치비교 연산자 `==`이다.
동치비교 연산자는 동일 유형의 값에 대해서만 작동하며, 서로 유형이 다른 값의 동치비교는 불가능하다.
원칙적으로는 모든 유형 각각에 대해 동치비교 연산자가 필요하지만
`==` 기호가 가능 모든 유형에 대해 동치비교 역할을 수행한다.

In [6]:
2 == 2

True

In [7]:
1.2 == 1.2

True

In [8]:
(True, False) == (True, False)

True

In [9]:
"Haskell" == "haskell"

False

#### 예제: `elem` 함수

`elem`은 리스트의 항목인지 여부를 판단해주는 함수이다. 
예를 들어, 정수들의 리스트에 특정 정수가 포함되어 있는지 여부를 아래와 같이 확인한다. 

**주의:** 두 개의 인자를 받는 함수를 중위표기법으로 사용하려면 역따옴표로 감싸면 된다.

In [10]:
1 `elem` [1, 2, 3]

True

또한 문자열에 특정 문자가 포함되어 있는지 여부도 확인이 가능하다.
이렇듯 `elem` 함수는 항목여부를 판단하는 기능을 다양한 유형의 리스트에 대해 동일하게 작동한다.

In [11]:
'a' `elem` "abc"

True

그런데 이렇게 동일한 이름의 함수나 기호를 임의의 유형에 대해서 사용할 수 있지는 않다.
예를 들어, 덧셈 연산자는 정수, 부동소수점 등 특정 유형들에 대해서 작동하지만,
`True + False`, `[1, 2] + [0, 3]`의 값은 아예 정의되어 있지 않다.
또한 동치비교 연산의 경우, `Float -> Float` 유형의 함수,
즉, 실수에서 실수로 가는 두 함수의 동치성 판단은 일반적으로 불가능하다.

이렇듯 몇몇 특정 유형들로 한정해서 동일한 이름(또는 기호)의 함수를 사용하는 것을 
**중복정의** 또는 **오버로딩**overloading이라 부른다.
중복정의는 하나의 함수가 여러 유형들과 함께 적용될 수 있다는 의미에서 다형성과 밀접하게 관련되어 있다.
다만 허용되는 유형이 제한된다는 의미에서 매개변수 다형성과 다르며, 일명 **도구 다형성**이라고도 한다.

도구 다형성의 영어 표현은 ad hoc polymorphism이며, 보통 애드혹 다형성으로 번역되지만,
여기서는 **함수가 갖는 기능의 다형성**이라는 측면을 강조하하기 위해 도구 다형성이란 표현을 사용한다.

## 유형클래스

앞서 언급한 단어 'ad hoc'의 사전적 의미는 '특별히', '당면 문제에 한해서', '임시 변통의' 등이지만 
여기서는 **함수의 기능이 특정 유형들에 제한된다**의 의미로 이해해야 한다.
따라서 특정 함수가 주어졌을 때 해당 함수의 기능이 발휘될 수 있는 유형들의 모음을 살펴보는 일이 중요하며,
그런 유형들의 모음을 **유형클래스typeclass**라고 부른다.

유형클래스는 몇몇 **함수들의 명세**specification와 그 **함수들이 가져야 하는 속성**들에 의해 정의된다.
그리고 특정 유형클래스에 속한 모든 함수들의 기능을 지원하는 유형을 해당 유형클래스의 
**인스턴스**instance라고 부른다.
즉, 유형클래스는 자신의 인스턴스가 되는 유형들의 집합에 대응한다.

정리하면 다음과 같다.

* 유형클래스: 유한 개의 함수와 함수들의 속성들로 구성된 하나의 집합으로 정의됨.
* 인스턴스: 유형클래스 정의에 사용된 모든 함수들의 기능을 지원하는 유형

특정 유형클래스 정의에 포함된 함수를 **메서드methods**라 부른다.
모든 메서드는 해당 유형클래스의 인스턴스 유형에 대해 동일한 이름으로 사용될 수 있다.
이런 의미에서 유형클래스를 메서드들의 중복정의 또는 오버로딩 기능과 동일시하여 설명하기도 한다.

유형클래스의 엄밀한 정의는 유형이론을 다루어야 하기에 좀 어렵다.
여기서는 하스켈에서 기본으로 제공하는 유형클래스 몇 개를 살펴보면서 유형클래스의 의미와 활용을 살펴볼 것이며,
사용자가 직접 유형클래스를 정의하는 방법은 추후에 다룰 예정이다. 

### 클래스와 객체 지향 프로그래밍

**주의: 객체지향 프로그래밍과 친숙한 경우만 읽으세요. 
친숙하지 않다면 괜히 혼란스러워질 수 있기 때문에 가급적 읽지 말고 
다음 "하스켈 주요 유형클래스" 설명으로 넘어가는 것을 권유합니다.**

C++, C#, 자바, 파이썬, 스칼라 등 명령형 객체지향프로그래밍 언어를 사용해보았다면
클래스와 인스턴스(객체) 개념이 친숙하게 다가올 것이다. 
실제로 하스켈 언어와 기타 객체지향 언어에서 지원하는 클래스와 인스턴스 개념이 
근본적으로 다르지 않다. 
다만 사용법에 있어서 차이가 좀 있을 뿐이다.
물론 언급된 언어들도 클래스와 인스턴스의 사용법이 모두 제각각 다른 특성을 갖는다.

하지만 하스켈에서 제공하는 클래스와 인스턴스 개념과 사용법이 가장 기본적이다. 
하스켈의 클래스와 인스턴스 개념을 이해한다면 기타 언어의 클래스와 인스턴스 개념을,
더 나아가 객체지향 프로그래밍의 개념을 보다 잘 이해할 수 있을 것이다.
물론 역으로도 가능하다. 다만, 앞서 언급한대로 차이점에 주의하면서 살펴보아야 한다.

하스켈의 클래스와 인스턴스의 개념과 가장 비슷한 것은 
자바 언어에서 제공되는 인터페이스interface와 implement 개념이다.
자바의 인터페이스 특징은 다음과 같다.

* 인터페이스: 추상 메서드abstract method들으로만 구성됨
* 인터페이스 구현implementation: 인터 페이스에 포함된 메서드(함수) 전부를 구체적인 함수들로 구현한 클래스

따라서 자바의 인터페이스는 하스켈의 유형클래스에,
자바의 인터페이스 구현은 하스켈의 인스턴스 정의에 대응된다.
하지만 앞서 설명한 대응이 절대적이지는 않다.
가장 큰 차이점은 다음과 같다.

* 하스켈의 유형은 특정 함수의 기능과 기본적으로 상관 없다. 
    즉, 유형이 일종의 집합으로 먼저 선언되고 이후에 특정 유형클래스의 함수들, 즉 메서드들을 구현하면서
    해당 유형클래스의 인스턴스로 선언된다.
* 자바의 인터페이스 구현은 집합과 메서드들의 구현을 동시에 해야 하며, 메서드가
    구현된 클래스의 주요 구성요소가 된다.
    
반면에 주요 공통점은 다음과 같다.

* 하스켈에서 하나의 유형이 동시에 여러 유형클래스의 인스턴스가 될 수 있다.
    이는 자바에서 여러 개의 인터페이스를 동시에 구현하는 클래스를 정의할 수 있는 것에 해당한다.
* 하스켈에서 유형클래스들 간의 다중상속이 가능하며,
    이는 자바에서 인터페이스들 간의 다중상속이 가능한 것에 해당한다.

## 하스켈 주요 유형클래스

하스켈의 기본 모듈인 `Prelude`에 포함된 많은 유형들 중에서 가장 많이 사용되는 유형클래스들을 살펴본다. 

### `Eq` 유형클래스

동치비교가 가능한 인자들만을 갖는 유형들을 인스턴스로 갖는 유형클래스를 `Eq` 라고 부른다.
먼저 `Eq`에 포함된 메서드들의 명세는 다음과 같다.

```haskell
(==) :: a -> a -> Bool
(/=) :: a -> a -> Bool
```

여기서 `a`는 유형 변수이며, 특정 유형 `A`가 `Eq` 클래스의 인스턴스가 되려면 
`(==) :: A -> A -> Bool`와 `(/=) :: A -> A -> Bool` 두 함수를 구체적으로 구현해야 한다.
또한 구현된 두 함수가 아래 성질을 만족시켜야 한다.

```haskell
x == y = not (x /= y)
x /= y = not (x == y)
```

즉, `x == y`와 `x /= y`는 서로 반대의 값을 가져야 한다. 

하스켈 문서에서 `Eq` 유형클래스에 대한 정보는 
[Prelude:Eq](https://hackage.haskell.org/package/base-4.12.0.0/docs/Prelude.html#t:Eq)에서
확인할 수 있다.

`GHCi`에서는 `:info` 명령을 통해 유형클래스의 정보를 확인할 수 있다.

In [12]:
:info Eq

위 명령을 실행하면 다음과 같이 보여준다.

```haskell
class Eq a where  
    (==) :: a -> a -> Bool  
    (/=) :: a -> a -> Bool  
    {-# MINIMAL (==) | (/=) #-}
...
```

여기서 `{-# MINIMAL (==) | (/=) #-}`의 의미는 `(==)` 또는 `(/-)` 둘 중의 하나만 구현하면 된다는 뜻이다.
두 연산자 중의 하나를 구현하면 `==` 와 `/=`의 성질에 의해 다른 연산자의 정의는 자연스럽게 따라온다. 
그리고 두 연산자 모두 따로 정의하면 서로 반대의 값을 가지도록 해야 한다. 
그렇게 하지 않으면 오류가 발생한다. 

#### `(==)`와 `(/=)` 연산자의 유형

앞서 설명하였듯이, `(==)`와 `(/=)`는 `Eq` 유형클래스의 인스턴스에 대해서만 작동한다.
두 함수의 유형에 위 사실이 내포되어 있다.

In [13]:
:t (==)

(==) :: forall a. Eq a => a -> a -> Bool

In [14]:
:t (/=)

(/=) :: forall a. Eq a => a -> a -> Bool

위 유형의 의미는 다음과 같다.

* `Eq A`를 만족시키는 유형 `A`, 즉 `Eq` 유형클래스의 인스턴스 `A`가 주어졌고,
* `x`, `y`가 모두 `A` 유형의 값이면 
* `x == y`와 `x /= y` 모두 `Bool` 유형의 값, 즉, `True` 아니면 `False` 이다.

#### `Eq` 유형클래스의 인스턴스

지금까지 살펴본 유형들 중에서 함수 유형을 제외한 다른 유형들 모두 `Eq` 유형클래스의 인스턴스이다. 
예를 들어, `Bool`, `Char`, `String`, `Int`, `Integer`, `Float`, `Double` 뿐만 아니라,
리스트, 튜플 유형들 모두 `Eq` 유형클래스의 인스턴스이다. 
다만, 리스트와 튜플의 경우 항목들의 유형 모두 `Eq` 유형클래스의 인스턴스이어야 한다.

##### 참조: 함수의 동치비교 불가능성

`A`와 `B` 모두 `Eq` 유형클래스의 인스턴스이더라도, 
함수 유형 `A -> B`는 일반적으로 `Eq` 유형클래스의 인스턴스가 아니다.
두 함수의 동치비교는 일반적으로 불가능하다. 
만약에 가능하다면 튜링의 정지문제가 해결가능하게 되는데 이는 불가능함이 이미 증명되었다. 
정지문제와 함수의 동치비교 불가능성에 대한 설명은 아래 내용을 추천한다.

* 정지문제 설명: [유튜브: Proof That Computers Can't Do Everything](https://www.youtube.com/watch?time_continue=308&v=92WHN-pAFCs&feature=emb_logo)
    * 한글자막 지원됨.
* 함수 동치비교 불가능성 설명: [스택 오버플로우: Is finding the equivalence of two functions undecidable?](https://stackoverflow.com/questions/1132051/is-finding-the-equivalence-of-two-functions-undecidable)

In [15]:
True == not True

Line 1: Redundant ==
Found:
True == not True
Why not:
not TrueLine 1: Evaluate
Found:
not True
Why not:
False

False

In [16]:
1 == 2

False

In [17]:
'a' == 'c'

False

In [18]:
"abc" == "abc"

True

In [19]:
[1, 2, 3] == [1, 2, 3]

True

In [20]:
(1, 'a') == (2, 'a')

False

In [21]:
1.1 == 1.10

True

In [22]:
1.1 == 1.101

False

하지만 이전에 언급한대로 부동소수점은 매우 조심스럽게 다뤄야 한다.
실제로 아래와 같은 일이 발생한다. 
끝자리에 있는 1에 주의하라.

In [23]:
1.1 == 1.1000000000000001

True

이런 현상은 하스켈의 문제가 아니라 부동소수점을 사용하는 모든 컴퓨터의 문제이다.

#### `elem` 함수: `Eq` 유형클래스 활용

앞서 살펴본 `elem` 함수는 리스트의 항목인지 여부를 판단하기 위해
항목에 포함된 인자들과의 동치비교를 할 수 있어야 한다.

아래 예제를 살펴보자.

```haskell
x `elem` [x1, x2, x3, x4]
```

위 표현식이 의미를 가지려면 `x`와 `xi`들이 모두 동일한 유형의 값이어야 하며,
또한 `x`와 `xi`들 사이의 동치비교, 즉 `x == xi`가 의미를 가져야 한다.
즉, `x, x1, x2, x3, x4` 의 유형이 `Eq`의 인스턴스 이어야 한다.

따라서 `elem` 아래와 같은 형식의 유형을 가져야 한다. 

```haskell
elem :: forall a. (Eq a) => a -> [a] -> Bool
```

`enum` 함수의 실제 유형은 다음과 같다.

In [24]:
:t elem

elem :: forall (t :: * -> *) a. (Foldable t, Eq a) => a -> t a -> Bool

**주의:** `Foldable`은 일단 무시한다. 여기서는 `t a`를 `[a]`로 이해하는 정도로 하고 넘어간다.

### `Ord` 유형클래스

`Ord` 유형클래스는 `Eq`를 확장하며, 
`(==)`와 `(/=)` 이외에 크기비교와 관련된 7개의 메서드에 대한 명세가 추가된다. 

```haskell
class Eq a => Ord a where
    compare :: a -> a -> Ordering
    (<) :: a -> a -> Bool
    (<=) :: a -> a -> Bool
    (>) :: a -> a -> Bool
    (>=) :: a -> a -> Bool
    max :: a -> a -> a
    min :: a -> a -> a
    {-# MINIMAL compare | (<=) #-}
...
```

#### 유형클래스 상속

`Eq a => Ord a`는 `Ord` 유형클래스가 `Eq` 유형클래스를 **상속**한다는 사실을 나타낸다.
즉, `Eq`에 포함된 메소드들 모두 `Ord`에 포함된다.

따라서 `class Eq a => Ord a where`의 의미는 다음과 같다.

* 유형 `A`가 `Ord` 유형클래스의 인스턴스가 되려면 먼저 `Eq` 유형클래스의 인스턴스이어야 하며,
* `Eq` 유형클래스의 명세에 포함되어 있는 `(==)`와 `(/=)` 함수 이외에 7개의 함수에 대한 명세와 성질을
    만족시켜야 한다. 

하지만 `{-# MINIMAL compare | (<=) #-}`가 의미하는 대로  `compare` 또는 `(<=)` 두 함수중 하나만
정의하면 된다. 이유는 `(==)`와 `(/=)`의 관계처럼 다른 함수들은 
아래 성질들이 만족되도록 `compare` 또는 `(<=)`에 의해 정의될 수 있기 때문이다.

```
x >= y = y <= x
x < y = x <= y && x /= y
x > y = y < x
x < y = compare x y == LT
x > y = compare x y == GT
x == y = compare x y == EQ
min x y == if x <= y then x else y = True
max x y == if x >= y then x else y = True
```

하스켈 문서에서 `Ord` 유형클래스에 대한 정보는
[Prelude:Ord](https://hackage.haskell.org/package/base-4.12.0.0/docs/Prelude.html#t:Ord)
에서 확인할 수 있다. 
GHCi에서 `:info` 명령을 통해 확인하면 다음과 같다. 

####  `Ordering` 유형

`Ordering` 유형은 `LT`, `EQ`, `GT` 세 개의 값으로 이루어지며, 
세 값의 의미는 다음과 같다.

* `LT`: 작다(less than)
* `EQ`: 같다(equal)
* `GT`: 크다(greater than)

즉, 특정 유형을 아용하여 `compare` 함수를 구현하려면 해당 유형의 임의의 두 값을 비교한 후
두 값 사이의 순서ordering를 지정해야 한다. 

#### `Ord` 유형클래스의 인스턴스

하스켈에서 기본으로 제공하는 모든 유형은 모두 `Ord` 유형클래스의 인스턴스이다. 
다만, 리스트와 튜플의 경우 항목들의 유형 모두 `Ord` 유형클래스의 인스턴스이어야 한다.

예를 들어, 정수들 사이의 순서는 우리가 아는 것과 동일하다. 

In [25]:
compare 2 1

GT

In [26]:
2 <= 2

True

부울값들도 알려진 바와 동일하다.

In [27]:
False < True

True

문자 순서는 알파벳 순서를 따른다.

In [28]:
compare 'a' 'b'

LT

리스트와 튜플의 비교도 사전식으로 작동한다.

In [29]:
compare [1, 5] [2, 3, 4]

LT

In [30]:
[1, 5] <= [2, 3, 4]

True

In [31]:
max ('b', 3) ('b', 0)

('b',3)

문자열은 문자들의 리스트임에 주의하라.

In [32]:
"cd" > "aef"

True

In [33]:
min "cd" "aef"

"aef"

### `Show` 유형클래스

`Show` 유형크래스의 인스턴스가 되어야 해당 유형의 값들을 화면에 출력print시킬 수 있다. 
하스켈에서 기본으로 제공하는 모든 유형은 모두 `Show` 유형클래스의 인스턴스이다. 
다만, 리스트와 튜플의 경우 항목들의 유형 모두 `Show` 유형클래스의 인스턴스이어야 한다.

`Show` 유형클래스의 명세는 다음과 같다. 

```haskell
class Show a where
    showsPrec :: Int -> a -> showS
    show :: a -> String
    showList :: [a] -> ShowS
    {-# MINIMAL showsPrec | show #-}
...
```

즉, `showsPrec` 또는 `show` 중에 하나만 정의하면 되며, 
`show :: a -> String` 함수가 유형 `a`의 값들을 어떻게 화면에 출력하는지를 결정한다. 

**주의:** `showsPrec`과 `showList` 함수는 여기서 설명하기에는 좀 복잡하다. 

`show` 함수의 활용 예제는 다음과 같다.

In [34]:
show 3

"3"

In [35]:
show 1.345

"1.345"

In [36]:
show True

"True"

In [37]:
show [1.1, 2.0, 3.3]

"[1.1,2.0,3.3]"

문자열의 출력에 주의하라. 

* `\"` 는 큰따옴표를 특수기호가 아닌 인용부호 기호문자 `"` 그대로 다룬다는 것을 의미한다.

In [38]:
show "abc"

"\"abc\""

In [39]:
show ('a', "abc", False)

"('a',\"abc\",False)"

### `Read` 유형클래스

`Read` 유형크래스의 인스턴스가 되면 문자열로 변환된 해당 유형의 값을 `read` 함수를 이용하여 
원래 유형의 값으로 되돌릴 수 있다. 

```haskell
read :: String -> a
```

**참조:** 파이썬의 `eval()` 함수가 유사한 기능을 제공한다.

하스켈에서 기본으로 제공하는 모든 유형은 모두 `Read` 유형클래스의 인스턴스이다. 
다만, 리스트와 튜플의 경우 항목들의 유형 모두 `Read` 유형클래스의 인스턴스이어야 한다.

`Read` 유형클래스의 명세는 다음과 같다. 

```haskell
class Read a where
    readsPrec :: Int -> ReadS a
    readList :: ReadS [a]
    readPrec :: ReadPrec a
    readListPrec :: ReadPrec [a]
    {-# MINIMAL readsPrec | readPrec #-}
...
```

**주의:** `Read` 유형클래스의 명세에 사용된 메서드들을 여기서 설명하기에는 좀 복잡하다.
다만 `read :: String -> a` 함수가 `readsPrec` 또는 `readPrec`에 의해 정의되고
문자열을 지정된 유형의 값으로 변환한다는 정도만 기억하면 된다.

또한 `read` 함수가 계산한 값의 유형을 지정해야 한다는 점에 유의해야 한다.
`read` 함수의 정의에 따르면 계산된 결과가 어떤 유형의 값인가를 미리 알 수는 없기 때문이다.
예를 들어 `read "1"` 의 유형은 `Int` 또는 `Integer` 둘 중에 하나이며,
어느 유형인지를 `read "1"` 의 형식만 보아서는 알 수 없다. 

`read` 함수의 활용 예제는 다음과 같다.


In [40]:
read "False" :: Bool

False

In [41]:
read "'a'" :: Char

'a'

In [42]:
read "\"Hello Haskell!\"" :: String

"Hello Haskell!"

하지만 경우에 따라 유형을 지정할 필요가 없다. 
유형의 추론이 가능하기 때문이다.
아래 예제의 경우 `||` 가 부울 유형의 논리합(logical or) 연산자이기 때문에
`read "False"` 유형은 `Bool` 이어야 한다. 

In [43]:
read "False" || True

True

아래의 경우들도 마찬가지이다.

In [44]:
not (read "False")

True

In [45]:
read "2" / 4

0.5

물론 오류가 발생할 수도 있다.
이유는 `read "2"`가 절대로 `Bool` 유형의 값이 될 수 없기 때문이다.

In [46]:
not (read "2")

: 

### `Num` 유형클래스

`Num` 유형클래스는 **수number**와 관련된 유형이 가져야 하는 **기본적인 메서드들의 명세**를 담고 있다.
`Int`, `Integer`, `Float`, `Double` 모두 `Num` 유형클래스의 인스턴스들이다. 

```haskell
class Num a where
    (+) :: a -> a -> a
    (-) :: a -> a -> a
    (*) :: a -> a -> a
    negate :: a -> a
    abs :: a -> a
    signum :: a -> a
    fromInteger :: Integer -> a
    {-# MINIMAL (+), (*), abs, signum, fromInteger, (negate | (-)) #-}
...
```

`Num` 유형클래스의 인스턴스가 되려면 아래 7개의 함수 중에 최소 6개를 구현해야 한다. 

* `(+)`: 덧셈 연산자
* `(*)`: 곱셈 연산자
* `(-)`: 뺄셈 연산자
* `negate`: 부호 전환 연산자, 즉, 음수는 양수로, 양수는 음수로 전환시키는 함수.
    * 보통 단항 연산자 기호인 `-`로 표기됨.
    * 즉, `negate x = -x`
* `abs`: 절댓값 연산자
* `signum`: 음수, 0, 양수를 가리키는 부호 연산자
* `fromInteger`: `Integer`, 즉 정수에 해당하는 수를 지정하는 함수

단, 아래 조건을 만족시켜야 한다. 

```haskell
-x = 0 - x
abs x * signum x == x
```

**주의:** `fromInteger` 함수로 인해 `Num` 유형클래스의 모든 유형이 정수들의 집합을 
부분집합으로 가져야 한다고 오해할 수 있다. 하지만 전혀 그럴 필요가 없다. 
예를 들어, 고정정밀도를 사용하는 `Int`가 비록 `Num` 유형클래스의 인스턴스이기는 하지만
무한정밀도를 사용하는 `Integer`를 부분집합으로 가질 수 없다. 
그리고 `Natural`이라고 자연수들의 집합에 해당하는 유형이 존재하며 역시 `Num` 유형클래스의 인스턴스이다. 
이 경우 `fromInteger` 함수는
음수는 무시한다. 실제로 `fromInteger -1` 등은 오류를 발생시키도록 정의되었다. 

In [47]:
negate 3.1

-3.1

In [48]:
- 3.1

-3.1

In [49]:
negate 3 + 3

0

In [50]:
3 + negate 3

0

`negate` 대신에 `-` 기호를 사용할 경우 괄호에 주의해야 한다. 


In [51]:
-3 + 3

0

아래와 같이 하면 파싱 오류가 발생한다. 
`-` 기호를 `negate`로 해석할지, 아니면 뺄셈으로 해석할지 확실하지 않기 때문이다.

In [52]:
3 + - 3

: 

**참조:** 파싱은 작성된 소스코드가 컴퓨터가 이해할 수 있는 언어로 컴파일되는 과정에서 
컴파일러가 작성된 코드의 구문을 분석하기 위해 사용하는 기법이다.
컴파일러, 파싱, 파서 등에 대한 간략한 소개는 
[컴파일러 만들기](https://crystalcube.co.kr/107?category=461606)를 참조할 수 있다.

In [53]:
abs (-2)

2

In [54]:
abs (-2) * signum (-2) == -2

True

In [55]:
signum (-3.1)

-1.0

In [56]:
signum 2.3

1.0

#### 예제: 숫자

1, 2, 3, ... 등의 정수, 1.0, 2.3, 3.8 등의 부동소수점도 `Num` 유형클래스의 모든 인스턴스에
포함되어 있다. 
즉, 중복으로 정의되어 있으며, 우리에게 보이는 숫자들이 특정 유형에서 공통으로 사용되는 
**상수constant** 기호라는 의미이다.

In [57]:
:t 1

1 :: forall p. Num p => p

부동소수점은 `Num`을 확장하는 `Fractional` 유형클래스에 속하는 유형에서 공통으로 사용되는 상수 기호이다. 
`Float`와 `Double`은 `Fractional`의 인스턴스이다.

In [58]:
:t 3.1

3.1 :: forall p. Fractional p => p

### `Fractional` 유형클래스

`Num` 유형클래스는 나눗셈에 대한 명세를 포함하지 않는다.
따라서 정수와 부동소수점을 유형클래스로 구분하기 위해 
`Num`을 상속하면서 나눗셈 함수에 대한 명세를 추가하여 새로운 유형클래스인 `Fractional`을 정의한다.

**참조:** `fraction`의 사전적 의미에 **분수**가 포함되어 있다. 
컴퓨터에서 사용되는 부동소수점은 분수로 표기될 수 있는 유리수이다. 

```haskell
class Num a => Fractional a where
    (/) :: a -> a -> a
    recip :: a -> a
    fromRational :: Rational -> a
    {-# MINIMAL fromRational, (recip | (/)) #-}
...
```

* `(/)`: 나눗셈 연산자에 해당
* `recip`: 역수reciprocal 계산 연산자에 해당.
* `fromRational`: `Num` 유형클래스에 포함된 `fromInteger`와 유사한 기능을 수행.
    다만, `Integer` 대신에 `Rational`, 즉, 정수들을 이용한 분수들의 집합에 해당하는 유형이 사용됨.

이미 언급한 대로 `Float`와 `Double`은 `Fractional`의 인스턴스이다. 
반면에 `Int`와 `Integer`는 그렇지 않은데,
이는 정수들의 나눗셈은 일반적으로 정수가 아니라는 사실에 부합한다.

In [59]:
3.5 / 2.0

1.75

In [60]:
recip 4.0

0.25

## 다중상속

하스켈 유형클래스는 다중상속을 지원한다. 
다중상속이 사용된 기본 유형클래스 몇 개를 소개한다. 

### `Integral` 유형클래스

`Integral`은 정수 나눗셈의 몫과 나머지 연산자를 제공하며, 
`Real`과 `Enum` 두 유형클래스를 상속하며, 
추가되는 메서드 중에 `div`와 `mod`가 대표적이다.


```haskell
class (Real a, Enum a) => Integral a where
    div :: a -> a -> a
    mod :: a -> a -> a
...
```

`Integral` 유형클래스에 포함된 두 메서드 `div`와 `mod`는 
각각 정수 나눗셈의 몫과 나머지 연산자를 가리키며, 아래 성질을 만족시킨다.

```haskell
(x `div` y) * y + (x `mod` y) == x
```

In [61]:
10 `div` 3

3

In [62]:
10 `mod` 3

1

### `Real` 유형클래스

`Real` 유형클래스의 인스턴스는 소위 실수집합의 부분집합으로 간주될 수 있는 유형들이며, 
`Int`, `Integer`, `Float`, `Double` 등이 포함된다.

실제 정의는 아래와 같으며 `Num`과 `Ord`를 상속한다. 

```haskell
class (Num a, Ord a) => Real a where
   toRational :: a -> Rational
...
```

### `Enum` 유형클래스

`Enum` 유형클래스의 인스턴스는 소위 `+1`과 `-1` 연산자가 정의될 수 있는 유형들이며,
`Char`, `Bool`, `Int`, `Integer`, `Float`, `Double` 등이 포함된다.    

실제 정의는 아래와 같다.

```haskell
class Enum a where
    succ :: a -> a
    pred :: a -> a
...
```

* `succ` 메서드: `+1`에 해당
* `pred` 메서드: `-1`에 해당

## 표현식과 유형과 종과 유형클래스의 관계

**주의: 아래 내용은 좀 어려움. 세세한 설명은 무시하고 결론만 기억해 두면 좋음.**

하스켈에서 사용되는 모든 대상은 유형을 갖는다. 
문자, 문자열, 숫자, 리스트, 튜플, 그리고 함수까지 모두 유형을 갖는다는 것은 이미 확인하였다.
그런데 그런 값들의 유형도 하스켈에서 다루는 대상이기 때문에 유형을 가져야 한다. 
예를 들어, `Integer`의 유형은 무엇이고, `Char`의 유형은 무엇인가?
또 `[Bool]`과 `Int -> Double`의 유형은 무엇인가?

이 질문에 대답하려면 하스켈에서 사용되는 대상들을 세 부류로 나눈다는 것을 알아야 한다.

1. 표현식expression: `1`, `True`, `1.1 + 2.0`, `[1, 2, 3]`, `(+)`, `abs` 등 
    계산을 할 수 있는 대상들로 구성되는 부류
1. 유형type: `Int`, `Bool`, `Float`, `[Integer]`, `forall a. Num a => a -> a -> a`,
    `forall a. Num a => a -> a` 등 표현식의 유형들로 구성되는 부류
1. 종kind: 유형type의 유형들로 구성되는 부류
    * `*`, `* -> *`, `* -> Constraint` 등등

### 표현식의 유형

지금까지 살펴본 것은 표현식의 유형이다. 
`(+)`의 경우처럼 다형유형도 있지만 모든 표현식은 유형을 갖는다.

### 유형의 종

유형은 종kind를 자신들의 유형으로 갖는다. 바로 위에서 언급된 모든 유형들의 종은 `*`이다.

그리고 바로 위에서는 언급되지 않았지만 지금까지 살펴 본 것중에 
`* -> *`와 `* -> Constraint` 등을 종으로 갖는 유형도 포함되었다. 

#### 예제: 리스트 유형 생성 함수 `([])` 의 종

리스트에 포함되는지 여부를 확인해주는 함수 `elem`의 유형은 다음과 같다.

In [63]:
:t elem

elem :: forall (t :: * -> *) a. (Foldable t, Eq a) => a -> t a -> Bool

`elem`의 유형에서 다음 사실을 알 수 있다.

* `Foldable` 유형클래스의 인스턴스로 선언될 수 있는 유형의 종이 바로 `* -> *`이다.

앞서 `t a`를 `[a]`로 이해하고 넘어가라고 언급하였는데, 
임의의 유형 `a`를 이용하여 리스트 유형 `[a]`를 생성하는 함수 
`([])`가 `Foldable`의 대표적인 인스턴스이기 때문이다.

실제로 `([])`의 종kind을 확인하면 다음과 같다.

In [64]:
:kind ([])

([]) :: * -> *

#### 예제: 유형클래스 `Num` 의 종

`Num`은 아래와 같이 정의된다.

```haskell
class Num a where
    (+) :: a -> a -> a
    (-) :: a -> a -> a
    (*) :: a -> a -> a
    negate :: a -> a
    abs :: a -> a
    signum :: a -> a
    fromInteger :: Integer -> a
    {-# MINIMAL (+), (*), abs, signum, fromInteger, (negate | (-)) #-}
...
```

위 정의를 아래와 같이 이해할 수 있다.

* `Num`은 임의의 유형 `a`에 적용하면 `a` 유형이 가져야 하는 메서드와 관련 성질들을 산출한다. 

위 설명을 받아들인다면 `Num`의 종이 아래와 같음을 이해할 수 있다. 

In [65]:
:kind Num

Num :: * -> Constraint

`Num`의 종으로 언급된 `* -> Constraint`의 뜻은 다음과 같다.

* `*`를 종으로 갖는 임의의 유형 `a`를 인자로 받는다.
* 그러면 `Num a`는 `Constraint`를 종으로 갖는다.

여기서 `Constraint`는 **유형클래스의 정의에 사용된 메서드들의 명세와 관련된 성질들이 갖는 종**으로 이해하면 된다.

이 설명의 또다른 의미는 다음과 같다.

* 유형클래스 역시 일종의 유형으로 간주할 수 있다.

**참조:** constraint 단어의 사전적 의미가 제약, 제한, 조건 등인데 메서드와 관련 성질들에 의해
인스턴스로 인정받을 수 있는 유형들을 제한한다는 의미와 잘 어울린다.

### 종의 유형 또는 종은?

"하스켈에서 다루는 모드 대상은 유형이 있다"라는 명제가 참인가라는 질문이 아직 완벽하게 
답이 되지 않았다. 
바로 종 부류에 속하는 `*`, `* -> *`, `* -> Constraint`의 유형 또는 종은 무엇인지 설명되지 않았다.

이에 대해 하스켈은 다음과 같이 답한다. 

In [66]:
:k *

* :: *

In [67]:
:k (* -> *)

(* -> *) :: *

In [68]:
import GHC.Exts (Constraint)

In [69]:
:k Constraint

Constraint :: *

In [70]:
:k (* -> Constraint)

(* -> Constraint) :: *

**결론:** 모든 종의 종은 `*`이다. 

조금은 허무한 결론일 수 있다.
하지만 여기에는 유형론에 대한 긴 시간동안의 연구가 내포되어 있으며, 더 이상의 설명은 어렵다.

## 하스켈의 유형론

**주의: 아래 내용 역시 좀 어려움. 그냥 한 번 읽어 보는 것에 의미를 두면 됨.**

### System FC

하스켈 언어가 기반으로 삼는 유형론type theory이 있으며,
최신 [GHC 언어의 특징](https://downloads.haskell.org/ghc/latest/docs/html/users_guide/lang.html)을
거의 다 다룰 수 있는 유형이론은 
[System FC](https://gitlab.haskell.org/ghc/ghc/blob/master/docs/core-spec/core-spec.pdf)이다. 

System FC에 대한 간략한 소개는 
[System FC: equality constraints and coercions](https://gitlab.haskell.org/ghc/ghc/wikis/commentary/compiler/fc)
을 참고하면 되며, 내용을 요약하면 다음과 같다.

* System FC: System F${}_\omega$ + GADT + Coercions
* System F${}_\omega$: 지라Girard의 2계 유형론인 System F를 고계 유형론으로 확장한 유형론
* GADT: 일반화된 대수적 자료형(Generalized Algebraic Datatypes)의 줄임말임.
    * 대수적으로 정의될 수 있는 유형들의 집합을 가리킴.
    * **대수적**의 엄밀한 정의는 복잡함.
    * `data` 키워드로 시작하며, 귀납적으로 유형을 정의함.
* Coercions: 형변환, 즉, 특정 유형의 값을 다른 유형의 값으로 암묵적으로 전환시키는 작업을 가리킴.
    예를 들어, 정수 1을 부동소수점 1.0으로 알아서 간주하는 기능을 가리킴.
    ```haskell
    1 + 2.1 = 3.1
    ```

System F와 System F${}_\omega$에 대한 설명은 
[Lambda Cube](https://en.wikipedia.org/wiki/Lambda_cube)를 참조하면 좋다.

### 표현식, 유형, 종의 구분

앞서 설명한 대로 하스켈에서는 표현식, 유형, 종을 문법적으로 구분한다. 
하지만 Coq, Agda, Lean 등 수학적 증명을 지원하는 증명보조기의 경우는 다르다.
언급된 증명보조기에서 사용되는 언어가 기반으로 삼는 유형론은 
기본적으로 마틴-뢰프Martin-L&ouml;f의 직관주의적 유형론인데,
거기서는 표현식, 유형, 종을 구분하지 않는다.